# 1- Web Scraping and Data Cleansing

Let's import the needed library to perform the scraping and to get the web data into proper data structures

In [65]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

Getting the wikipedia URL and doing the web scraping

In [12]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source= requests.get(url).text
soup=BeautifulSoup(source,'lxml')


In [13]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"Xm3WfgpAMMUAATIesy8AAADB","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":945633050,"wgRevisionId":945633050,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Article

We need to get the Toronto Neighborhood data from the HTML data within the wikipedia page 

In [14]:
table=soup.find('table')
print(table.prettify())
#df = pd.read_html(str(table))
#print(df)


<table class="wikitable sortable">
 <tbody>
  <tr>
   <th>
    Postcode
   </th>
   <th>
    Borough
   </th>
   <th>
    Neighbourhood
   </th>
  </tr>
  <tr>
   <td>
    M1A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M2A
   </td>
   <td>
    Not assigned
   </td>
   <td>
    Not assigned
   </td>
  </tr>
  <tr>
   <td>
    M3A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Parkwoods" title="Parkwoods">
     Parkwoods
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M4A
   </td>
   <td>
    <a href="/wiki/North_York" title="North York">
     North York
    </a>
   </td>
   <td>
    <a href="/wiki/Victoria_Village" title="Victoria Village">
     Victoria Village
    </a>
   </td>
  </tr>
  <tr>
   <td>
    M5A
   </td>
   <td>
    <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">
     Downtown Toronto
    </a>
   </td>
   <td>
    <a href="

We extract then the HTML table headlines to get our columns names 

In [15]:
table_headlines=[]

for th in table.find_all('th') :
    #print(th.text.strip())
    table_headlines.append(th.text.strip())
    #print(ths)
print(table_headlines)    

['Postcode', 'Borough', 'Neighbourhood']


We then extract each row from the HTML table

In [16]:
data=[]
data_row=[]

for row in table.find_all('tr'):
    
    for item in row.find_all('td'):
        
        data_row.append(item.text.strip())
        
    data.append(data_row)
    data_row=[]
#print(data[1:])#print(data)
data=data[1:]   
print(data[:30])

[['M1A', 'Not assigned', 'Not assigned'], ['M2A', 'Not assigned', 'Not assigned'], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Harbourfront'], ['M6A', 'North York', 'Lawrence Heights'], ['M6A', 'North York', 'Lawrence Manor'], ['M7A', 'Downtown Toronto', "Queen's Park"], ['M8A', 'Not assigned', 'Not assigned'], ['M9A', 'Etobicoke', 'Islington Avenue'], ['M1B', 'Scarborough', 'Rouge'], ['M1B', 'Scarborough', 'Malvern'], ['M2B', 'Not assigned', 'Not assigned'], ['M3B', 'North York', 'Don Mills North'], ['M4B', 'East York', 'Woodbine Gardens'], ['M4B', 'East York', 'Parkview Hill'], ['M5B', 'Downtown Toronto', 'Ryerson'], ['M5B', 'Downtown Toronto', 'Garden District'], ['M6B', 'North York', 'Glencairn'], ['M7B', 'Not assigned', 'Not assigned'], ['M8B', 'Not assigned', 'Not assigned'], ['M9B', 'Etobicoke', 'Cloverdale'], ['M9B', 'Etobicoke', 'Islington'], ['M9B', 'Etobicoke', 'Martin Grove'], ['M9B', 'Etobicoke', 'Princess Gar

We then create a pandas dataframe out of the extracted data

In [124]:
df = pd.DataFrame(data,columns=table_headlines) 
df.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Check the data for non existing boroughs


In [153]:
df['Borough'].value_counts()

Not assigned        77
Etobicoke           45
North York          38
Downtown Toronto    37
Scarborough         37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Name: Borough, dtype: int64

Let's check for the Neighbourhood data as well

In [154]:
df['Neighbourhood'].value_counts()

Not assigned                77
St. James Town               2
Runnymede                    2
Davisville North             1
CN Tower                     1
                            ..
Lawrence Heights             1
Kingsway Park South West     1
Guildwood                    1
Woodbine Gardens             1
Forest Hill North            1
Name: Neighbourhood, Length: 209, dtype: int64

Let's remove the rows with Borough having a 'Not assigned' value

In [155]:
clean_df=df[df['Borough']!='Not assigned']
clean_df['Borough'].value_counts()
clean_df.shape

(210, 3)

There are no Neighbourhood values let with a 'Not assigned' value, as we can see in the below output

In [156]:
clean_df['Neighbourhood'].value_counts()

St. James Town                           2
Runnymede                                2
Davisville North                         1
Bloordale Gardens                        1
Canada Post Gateway Processing Centre    1
                                        ..
Lawrence Heights                         1
Kingsway Park South West                 1
Guildwood                                1
Woodbine Gardens                         1
Forest Hill North                        1
Name: Neighbourhood, Length: 208, dtype: int64

We need to remove redundant postcodes and have one entry with one unique postcode gathering all the corresponding neighbourhoods, let's first have a look at the redundant postcodes

In [146]:
toronto_df=clean_df
uniquePC=(clean_df.Postcode.value_counts().rename_axis('UniquePC').reset_index(name='counts'))
(uniquePC.head(30))
#print(val_cnt[0:5])
redundant_PC=uniquePC[uniquePC['counts']!=1]
print(redundant_PC)
print("Total number of redundant PC entries: ",sum(redundant_PC['counts']))

   UniquePC  counts
0       M9V       8
1       M8Y       8
2       M5V       7
3       M4V       5
4       M9B       5
5       M8Z       5
6       M1V       4
7       M6M       4
8       M9R       4
9       M9C       4
10      M6K       3
11      M8X       3
12      M5T       3
13      M2J       3
14      M5J       3
15      M5H       3
16      M6L       3
17      M3H       3
18      M5R       3
19      M1L       3
20      M1C       3
21      M8V       3
22      M1T       3
23      M1M       3
24      M1P       3
25      M1E       3
26      M1K       3
27      M6P       2
28      M8W       2
29      M4X       2
30      M6H       2
31      M5M       2
32      M6J       2
33      M3J       2
34      M5P       2
35      M4T       2
36      M1N       2
37      M5X       2
38      M1R       2
39      M4L       2
40      M4B       2
41      M6N       2
42      M5B       2
43      M5K       2
44      M9M       2
45      M2L       2
46      M6A       2
47      M3C       2
48      M6R       2


Let's clean our dataframe: let's remove each redundant postcode and replace all of the redundancy by one postcode with all related neighbourhoods

In [157]:


# goes through each postcode that has multiple occurencies 
for pc in redundant_PC['UniquePC']:
    
    temp_df=clean_df[clean_df['Postcode']==pc]
    tempstr=''
    # concatenates the list of neighbourhoods for each instance of the postcode
    for neighs in (temp_df['Neighbourhood']):
        #print(neighs) 
        tempstr =tempstr + ", " + neighs #adds every neighbourhood related to the postcode
    #removes the first 2 characters: ', ' from the list of neighbourhoods
    endstring=(len(tempstr))
    tempstr=tempstr[2:endstring]
    # prepare the unique PC row with all neighbourhood data
    row_list=[pc,temp_df.iloc[0,1],tempstr]
    row_series=pd.Series(row_list,index=clean_df.columns)
    # deletes all rows associated with the postcode
    clean_df=(clean_df[clean_df['Postcode']!=pc])
    # adds the unique PC row to the dataframe
    clean_df=clean_df.append(row_series,ignore_index=True)
    


    

Let's do a sanity check on the size of the resulting dataframe we had 210 rows with 163 redundant entries corresponding to 56 postcodes, we shall endup with 210-163+56=103 rows 

In [158]:
clean_df.shape

(103, 3)

The row count is right! Let's visualize the resulting dataframe

In [169]:
clean_df.head(103)
#clean_df[clean_df['Borough']=='Scarborough'].head(20)

,Postcode,Borough,Neighbourhood
13,M1G,Scarborough,Woburn
17,M1H,Scarborough,Cedarbrae
20,M1J,Scarborough,Scarborough Village
38,M1S,Scarborough,Agincourt
40,M1W,Scarborough,L'Amoreaux West
44,M1X,Scarborough,Upper Rouge
53,M1V,Scarborough,"Agincourt North, L'Amoreaux East, Milliken, St..."
66,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
67,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
69,M1T,Scarborough,"Clarks Corners, Sullivan, Tam O'Shanter"


Let's do a sanity check on the size of the resulting dataframe we had 210 rows with 163 redundant entries corresponding to 56 postcodes, we shall endup with 210-163+56=103 rows 

In [160]:
clean_df.shape

(103, 3)

Thanks to the pandas grouby method all the above code can be avoided, my C coding background had to do something with above code;-) After some reading about pandas, I realized that there are method that can do the agregation of strings when grouping by, like we did in the course when we were summing columns values after a pandas groupby, let's try it out! 

In [170]:
toronto_df=toronto_df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index() 
#df.groupby(['name','month'])['text'].apply(','.join).reset_index()
print("shape of dataframe: ", toronto_df.shape)
toronto_df.head(103)
#toronto_df[toronto_df['Borough']=='Scarborough'].head(20)


shape of dataframe:  (103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


Toronto Borough&Neighbourhood dataframe is ready to be saved 

In [171]:
toronto_df.to_csv(r'toronto.csv',index=False,header=True)

# 2- Geo Tagging

importing the needed library

In [1]:
%pip install geocoder

import geocoder # import geocoder



Note: you may need to restart the kernel to use updated packages.


We will use the code template provided from the assignment to play with the geocoder and assess it

In [11]:
# initialize your variable to None
lat_lng_coords = None
postal_code ="M5G"
# loop until you get the coordinates
#while(lat_lng_coords is None):
g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
lat_lng_coords = g.latlng
print(lat_lng_coords)
#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

None


The While loop has been commented out,  not to make the notebook hang, the geocoder has not provided geocoordinates, we will revert to the other proposed method, via a CSV file.

In [ ]:
!wget -O Geospatial_Coordinates.csv https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv

In [1]:
%pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9686 sha256=dae518b0b92f6e7eac89c59f26f005d72c3c133a2199998e13f09d490a0941ef
  Stored in directory: c:\users\chidams\appdata\local\pip\cache\wheels\a1\b6\7c\0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
Note: you may need to restart the kernel to use updated packages.


In [1]:
import wget

url = "https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"

wget.download(url)

'Geospatial_Coordinates.csv'

In [48]:
geo_data=pd.read_csv('Geospatial_Coordinates.csv')


In [59]:

geo_data.head(103)
#print(geo_data.iloc[0,0],"Latitude: ",geo_data.iloc[0,1]," Longitude: ",geo_data.iloc[0,2])

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


Retrieving the toronto data from the csv file we saved in the previous section  

In [172]:
toronto_df=pd.read_csv('toronto.csv')

In [173]:
toronto_df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [176]:
print(len(clean_df))
z=np.zeros(len(clean_df))
toronto_df['latitude']=z
toronto_df['longitude']=z

103


In [177]:
for pc,lat,long in zip(geo_data['Postal Code'],geo_data['Latitude'],geo_data['Longitude']):
    position=toronto_df.iloc[:,0]==pc
    position=np.where(position)[0]
    toronto_df.iloc[position[0],3]=lat
    toronto_df.iloc[position[0],4]=long

In [178]:
toronto_df.head(103)

,Postcode,Borough,Neighbourhood,latitude,longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


ok... sounds goods but still pandas offers a merge/join method and we could get easily an intersection or inner join between the toronto dataframe and the geodata dataframe based on the postal code or postcode to get a data frame coombining all the info,

Let's try the pandas alternative and see how the code could be optimized from the for loop that has been used before

In [194]:
toronto_df=pd.read_csv('toronto.csv')
toronto_df=pd.merge(toronto_df,geo_data,left_on='Postcode',right_on='Postal Code',how='inner')
toronto_df.drop('Postal Code',axis=1,inplace=True)
toronto_df.head(103)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",43.739416,-79.588437


Saving the Geo Toronto data into a CSV file!


In [195]:
toronto_df.to_csv(r'geo_toronto.csv',index=False,header=True)

# Exploring and Clustering the Toronto Neighbourhoods



In this section, we will use the Foursquare API to explore neighborhoods in Toronto City. We will use the explore function to get the most common venue categories in each neighborhood, and then consider each of these categories as features to group the neighborhoods into clusters. we will be using the k-means clustering algorithm and rely on the Folium library to visualize the neighborhoods in Toronto City and their clusters.

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [303]:
import numpy as np # library to handle data in a vectorized manner
##!conda install -c conda-forge python-wget

import pandas as pd # library for data analsysis

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
#from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


let's get the Toronto Geo data we prepared from previous work 

In [304]:
neighborhoods=pd.read_csv("geo_toronto.csv")
neighborhoods.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [305]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


#### Preparing the neighbourhoods map of Toronto City

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [306]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create a map of Toronto with neighborhoods superimposed on top.

In [307]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Exploring Toronto Neighbourhoods 

first let's ready our foursquare API connection

In [308]:
CLIENT_ID = 'FZ2CDP4HVQ0PUEDNEAZDTJC0XQQINOTU5C5PYMYRJF1504YQ' # your Foursquare ID
CLIENT_SECRET = 'MYLQJWLMBQ3U4CH0ADX41EQDUM4FXF4GVJXDIN1NGOEKUUXK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FZ2CDP4HVQ0PUEDNEAZDTJC0XQQINOTU5C5PYMYRJF1504YQ
CLIENT_SECRET:MYLQJWLMBQ3U4CH0ADX41EQDUM4FXF4GVJXDIN1NGOEKUUXK



Get the first neighborhood in our dataframe.

In [309]:
print(neighborhoods.loc[0,'Neighbourhood'])

Rouge,Malvern


Let's get its latitude and longitude as these will be inputs for the GET request towards the Foursquare API 

In [310]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge,Malvern are 43.8066863, -79.19435340000003.


Now we can prepare the GET request URL

In [311]:
# type your answer here
radius=500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/explore?client_id=FZ2CDP4HVQ0PUEDNEAZDTJC0XQQINOTU5C5PYMYRJF1504YQ&client_secret=MYLQJWLMBQ3U4CH0ADX41EQDUM4FXF4GVJXDIN1NGOEKUUXK&ll=43.8066863,-79.19435340000003&v=20180605&radius=500&limit=100'

launching the GET request towards foursquare's API

In [312]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e72a67a0be7b4001b97e506'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.811186304500005,
    'lng': -79.18812958073043},
   'sw': {'lat': 43.8021862955, 'lng': -79.20057721926962}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'di

Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [313]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [314]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\Chidams\anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Wendy's,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


And how many venues were returned by Foursquare?

In [315]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


#### Exploring all the neighbourhoods in Toronto 

Let's create a function to repeat the same previous process to all the neighborhoods in Toronto

In [316]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [317]:


toronto_venues=getNearbyVenues(names=neighborhoods['Neighbourhood'],latitudes=neighborhoods['Latitude'],longitudes=neighborhoods['Longitude'])



Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

We get the following venues data:

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()

Let's check how many venues were returned for each neighborhood

In [266]:
toronto_venues.groupby(['Neighborhood']).count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",3,3,3,3,3,3
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",10,10,10,10,10,10
"Alderwood,Long Branch",9,9,9,9,9,9
"Bathurst Manor,Downsview North,Wilson Heights",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",26,26,26,26,26,26
Berczy Park,56,56,56,56,56,56


let's find out how many unique categories are they in this result set

In [318]:
print("There are {} unique categories within this result set".format(len(toronto_venues['Venue Category'].unique())) )


There are 268 unique categories within this result set


In [319]:
toronto_venues.groupby(['Venue Category']).count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Accessories Store,2,2,2,2,2,2
Afghan Restaurant,1,1,1,1,1,1
Airport,2,2,2,2,2,2
Airport Food Court,1,1,1,1,1,1
Airport Gate,1,1,1,1,1,1
Airport Lounge,2,2,2,2,2,2
Airport Service,2,2,2,2,2,2
Airport Terminal,2,2,2,2,2,2
American Restaurant,28,28,28,28,28,28


#### Analysing each neighbourhood within Toronto

In [324]:
#for item  in toronto_venues['Venue Category']:
#    print(item)
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
#for col in (toronto_onehot.columns):
#    print(col)
# add neighborhood column back to dataframe
#toronto_onehot['Neighbourhood'] = toronto_venues['Neighborhood'] 

#moves 'Neighbourhood' column at the first column position
toronto_onehot.insert(0,'Neighbourhood',toronto_venues['Neighborhood'] )
#toronto_onehot.head()
# move neighborhood column to the first column
#fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
#toronto_onehot = toronto_onehot[fixed_columns]
print([toronto_onehot.columns[-1]])
print(list(toronto_onehot.columns[:-1]))
toronto_onehot.head()

['Yoga Studio']
['Neighbourhood', 'Accessories Store', 'Afghan Restaurant', 'Airport', 'Airport Food Court', 'Airport Gate', 'Airport Lounge', 'Airport Service', 'Airport Terminal', 'American Restaurant', 'Antique Shop', 'Aquarium', 'Art Gallery', 'Arts & Crafts Store', 'Asian Restaurant', 'Athletics & Sports', 'Auto Workshop', 'BBQ Joint', 'Baby Store', 'Bagel Shop', 'Bakery', 'Bank', 'Bar', 'Baseball Field', 'Baseball Stadium', 'Basketball Stadium', 'Beach', 'Bed & Breakfast', 'Beer Bar', 'Beer Store', 'Belgian Restaurant', 'Bike Shop', 'Bistro', 'Boat or Ferry', 'Bookstore', 'Boutique', 'Brazilian Restaurant', 'Breakfast Spot', 'Brewery', 'Bridal Shop', 'Bubble Tea Shop', 'Building', 'Burger Joint', 'Burrito Place', 'Bus Line', 'Bus Station', 'Business Service', 'Butcher', 'Cafeteria', 'Café', 'Cajun / Creole Restaurant', 'Camera Store', 'Candy Store', 'Caribbean Restaurant', 'Cheese Shop', 'Chinese Restaurant', 'Chocolate Shop', 'Church', 'Climbing Gym', 'Clothing Store', 'Cocktail

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Rouge,Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Let's examine the dataframe size

In [325]:
toronto_onehot.shape

(2241, 269)

Let's group the data by neighbourhood and compute the category type frequency with the mean

In [333]:
toronto_grouped=toronto_onehot.groupby(['Neighbourhood']).mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Portuguese Restaurant,Poutine Place,Print Shop,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Social Club,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.0,0.01,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.02,0.0,0.03,0.0,0.0,0.0,0.0,0.

In [337]:
toronto_grouped.shape

(99, 269)

Let's print each neighborhood along with the top 5 most common venues

In [350]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index() #.T for the transpose of toronto_grouped
    
    temp.columns = ['venue','freq']
    
    temp = temp.iloc[1:]
 
    temp['freq'] = temp['freq'].astype(float)
    
    temp = temp.round({'freq': 2})
    
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.07
1       Restaurant  0.05
2  Thai Restaurant  0.04
3             Café  0.04
4       Steakhouse  0.03


----Agincourt----
                       venue  freq
0             Breakfast Spot  0.25
1             Clothing Store  0.25
2                     Lounge  0.25
3  Latin American Restaurant  0.25
4                      Motel  0.00


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                       venue  freq
0                 Playground  0.33
1                       Park  0.33
2                Coffee Shop  0.33
3          Accessories Store  0.00
4  Middle Eastern Restaurant  0.00


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
                  venue  freq
0           Pizza Place   0.2
1         Grocery Store   0.2
2  Fast Food Restaurant   0.1
3              Pharmacy   0.1
4          Liquor Store   0.1


----Alderwood,Long B

                 venue  freq
0          Pizza Place  0.25
1                  Pub  0.25
2                 Park  0.25
3  Japanese Restaurant  0.25
4  Monument / Landmark  0.00


----Guildwood,Morningside,West Hill----
                 venue  freq
0                  Spa  0.11
1  Rental Car Location  0.11
2       Medical Center  0.11
3   Mexican Restaurant  0.11
4    Electronics Store  0.11


----Harbord,University of Toronto----
        venue  freq
0        Café  0.16
1   Bookstore  0.05
2  Restaurant  0.05
3         Bar  0.05
4      Bakery  0.05


----Harbourfront----
            venue  freq
0     Coffee Shop  0.15
1          Bakery  0.07
2            Park  0.07
3             Pub  0.07
4  Breakfast Spot  0.04


----Harbourfront East,Toronto Islands,Union Station----
                venue  freq
0         Coffee Shop  0.12
1            Aquarium  0.05
2               Hotel  0.04
3                Café  0.04
4  Italian Restaurant  0.04


----High Park,The Junction South----
                ve

              venue  freq
0  Ramen Restaurant  0.09
1       Pizza Place  0.06
2  Sushi Restaurant  0.06
3              Café  0.06
4       Coffee Shop  0.06


----Willowdale West----
            venue  freq
0        Pharmacy  0.14
1     Pizza Place  0.14
2  Discount Store  0.14
3   Grocery Store  0.14
4         Butcher  0.14


----Woburn----
                       venue  freq
0                Coffee Shop  0.67
1          Korean Restaurant  0.33
2          Accessories Store  0.00
3  Middle Eastern Restaurant  0.00
4                      Motel  0.00


----Woodbine Gardens,Parkview Hill----
                  venue  freq
0           Pizza Place  0.18
1              Pharmacy  0.09
2    Athletics & Sports  0.09
3  Fast Food Restaurant  0.09
4              Bus Line  0.09


----Woodbine Heights----
            venue  freq
0    Skating Rink  0.25
1        Pharmacy  0.12
2  Cosmetics Shop  0.12
3            Park  0.12
4     Video Store  0.12


----York Mills West----
                       venue 

Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [351]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [393]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Restaurant,Thai Restaurant,Café,Bar,Steakhouse,Asian Restaurant,Pizza Place,Bakery,Sushi Restaurant
1,Agincourt,Clothing Store,Latin American Restaurant,Lounge,Breakfast Spot,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Distribution Center
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Park,Playground,Coffee Shop,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Grocery Store,Pizza Place,Liquor Store,Sandwich Place,Pharmacy,Fast Food Restaurant,Fried Chicken Joint,Beer Store,Discount Store,Distribution Center
4,"Alderwood,Long Branch",Pizza Place,Sandwich Place,Skating Rink,Gym,Pharmacy,Coffee Shop,Pub,Pool,Yoga Studio,Diner


#### Cluster the Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [394]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', axis=1) 
#toronto_grouped_clustering.head()

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[:]

array([0, 4, 1, 4, 0, 0, 4, 0, 0, 4, 0, 0, 4, 1, 0, 0, 1, 0, 4, 0, 0, 4,
       0, 4, 4, 4, 0, 0, 4, 0, 0, 0, 4, 4, 4, 4, 4, 4, 1, 0, 1, 2, 0, 0,
       0, 1, 4, 4, 4, 0, 0, 4, 0, 4, 4, 2, 4, 4, 1, 4, 4, 0, 1, 0, 0, 4,
       1, 0, 4, 0, 0, 1, 0, 1, 4, 4, 0, 0, 3, 4, 0, 0, 0, 0, 4, 4, 0, 4,
       4, 4, 0, 0, 1, 0, 0, 0, 4, 4, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [396]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', (kmeans.labels_))

toronto_merged = neighborhoods

# merge toronto_grouped with toronto neighbourhoods data we had at the beinining before the exploration 
# to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood',how="inner")

toronto_merged.head(100) # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353,4,Fast Food Restaurant,Print Shop,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Diner
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497,0,Moving Target,Bar,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Discount Store
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711,4,Rental Car Location,Moving Target,Spa,Electronics Store,Medical Center,Intersection,Mexican Restaurant,Bank,Breakfast Spot,Curling Ice
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0,Coffee Shop,Korean Restaurant,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,4,Caribbean Restaurant,Bakery,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Gas Station,Bank,Hakka Restaurant,Electronics Store,Eastern European Restaurant
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,3,Playground,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029,0,Discount Store,Department Store,Chinese Restaurant,Hobby Shop,Coffee Shop,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577,4,Bakery,Bus Line,Soccer Field,Park,Bus Station,Metro Station,Intersection,Ice Cream Shop,Cuban Restaurant,Coworking Space
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476,4,American Restaurant,Motel,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848,4,College Stadium,Skating Rink,General Entertainment,Café,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Diner,Drugstore


Finally, let's visualize the resulting clusters

In [414]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster+1), parse_html=True)
        
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

####  Analysis of the clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

In [ ]:
Cluster 1:

In [399]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Scarborough,0,Moving Target,Bar,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant,Discount Store
3,Scarborough,0,Coffee Shop,Korean Restaurant,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio
6,Scarborough,0,Discount Store,Department Store,Chinese Restaurant,Hobby Shop,Coffee Shop,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
18,North York,0,Clothing Store,Coffee Shop,Women's Store,Fast Food Restaurant,Bakery,Tea Room,Toy / Game Store,Shoe Store,Bank,Juice Bar
22,North York,0,Ramen Restaurant,Coffee Shop,Pizza Place,Café,Sushi Restaurant,Sandwich Place,Electronics Store,Steakhouse,Ice Cream Shop,Indonesian Restaurant
24,North York,0,Pharmacy,Pizza Place,Bank,Butcher,Coffee Shop,Grocery Store,Discount Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
27,North York,0,Beer Store,Gym,Restaurant,Coffee Shop,Asian Restaurant,Concert Hall,Chinese Restaurant,Sandwich Place,Bike Shop,Sporting Goods Shop
28,North York,0,Coffee Shop,Bank,Ice Cream Shop,Frozen Yogurt Shop,Bridal Shop,Sandwich Place,Restaurant,Diner,Middle Eastern Restaurant,Supermarket
29,North York,0,Falafel Restaurant,Metro Station,Caribbean Restaurant,Bar,Massage Studio,Miscellaneous Shop,Coffee Shop,Doner Restaurant,Donut Shop,Drugstore
34,North York,0,Intersection,Coffee Shop,Hockey Arena,Portuguese Restaurant,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop


Cluster 2:

In [408]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Scarborough,1,Park,Playground,Coffee Shop,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
23,North York,1,Park,Bank,Convenience Store,Flower Shop,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore
25,North York,1,Food & Drink Shop,Park,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
30,North York,1,Park,Airport,Other Repair Shop,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
40,East York,1,Park,Convenience Store,Coffee Shop,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
44,Central Toronto,1,Park,Swim School,Bus Line,Drugstore,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Fast Food Restaurant
48,Central Toronto,1,Park,Playground,Restaurant,Comic Shop,Dim Sum Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
50,Downtown Toronto,1,Park,Trail,Playground,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
64,Central Toronto,1,Park,Trail,Sushi Restaurant,Jewelry Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
74,York,1,Park,Women's Store,Market,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Electronics Store


Cluster 3:

In [409]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Etobicoke,2,Baseball Field,Locksmith,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
97,North York,2,Baseball Field,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Filipino Restaurant


Cluster 4:

In [411]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough,3,Playground,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant


Cluster 5:

In [413]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,4,Fast Food Restaurant,Print Shop,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Diner
2,Scarborough,4,Rental Car Location,Moving Target,Spa,Electronics Store,Medical Center,Intersection,Mexican Restaurant,Bank,Breakfast Spot,Curling Ice
4,Scarborough,4,Caribbean Restaurant,Bakery,Fried Chicken Joint,Athletics & Sports,Thai Restaurant,Gas Station,Bank,Hakka Restaurant,Electronics Store,Eastern European Restaurant
7,Scarborough,4,Bakery,Bus Line,Soccer Field,Park,Bus Station,Metro Station,Intersection,Ice Cream Shop,Cuban Restaurant,Coworking Space
8,Scarborough,4,American Restaurant,Motel,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
9,Scarborough,4,College Stadium,Skating Rink,General Entertainment,Café,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Diner,Drugstore
10,Scarborough,4,Indian Restaurant,Pet Store,Brewery,Vietnamese Restaurant,Chinese Restaurant,Donut Shop,Discount Store,Distribution Center,Dog Run,Doner Restaurant
11,Scarborough,4,Smoke Shop,Breakfast Spot,Bakery,Middle Eastern Restaurant,Yoga Studio,Dumpling Restaurant,Doner Restaurant,Donut Shop,Drugstore,Electronics Store
12,Scarborough,4,Clothing Store,Latin American Restaurant,Lounge,Breakfast Spot,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Distribution Center
13,Scarborough,4,Pharmacy,Pizza Place,Fast Food Restaurant,Gas Station,Fried Chicken Joint,Italian Restaurant,Chinese Restaurant,Thai Restaurant,Bank,Noodle House


Cluster 1: a quite big cluster, with a strong density around downtown Toronto, with a strong presence of cafés, bars, pubs and restaurants, this neighboorhoods cluster could relate to a young adult population who is thrilled by going out and having social evening activities - There are also hotels occurencies within this cluster and that supports the above findings as Tourists, business travelers are also attracted by areas where they can have the possibility to go outside in the evening. 

Cluster 2: a medium size type of cluster, we notice that it is located up north from the epicenter of cluster 1, we see that parks are a top feature within it, followed by playgrounds and a high frequency of distribution centers and drugstores, and dog runs, these characteristics are striking as the needs of a family with kids, we get a hint of that seeing that restaurants have a much lower frequency than say in cluster 1, this could be explained by the fact that families are not so picky about having restaurants for their every day life apart maybe in the weekends, these restaurants could be for the persons in transit.


Cluster 3: very small cluster with basically 2 neighbourhoods, motsly area with baseball fields and some places to eat maybe for people to  go  after the games have ended

Cluster 4: the smallest cluster, it looks like cluster 3 but is more for kids instead of baseball fans as its main attraction is a playground - and same characteristic of cluster 3, we have a plethoria of fast foods and restaurants for the population coming to the playground most probably.

Cluster 5:  A strong cluster, it reminds geographically of cluster 1 but it is more spread , we see there as well restaurants, fast foods,cafés but also shops and gym centers, bakeries, it could be areas for newly formed family, couples, looking for a tradeoff between an active evening life (that was the feature of cluster 1) and quality of life (feature of cluster 2)